# Modul 3: Hybrid-Workflow "Sieb + FJ32"

**Ziel:** Kombination des schnellen Numba-Siebs aus Modul 2 mit dem deterministischen FJ32-Test aus Modul 1

**Autor:** Sebastian
**Datum:** September 2025
**Repository:** C:\Users\sebas\Desktop\coding\PRIM

## Überblick

Dieses Notebook implementiert und testet einen Hybrid-Ansatz:
1. **Schnelle Primzahl-Generierung** mit dem optimierten Numba-Sieb
2. **Verifikation der Ergebnisse** mit dem deterministischen FJ32-Test
3. **Performance-Benchmark** der verschiedenen Ansätze
4. **Analyse und Visualisierung** der Ergebnisse

## 1. Setup und Imports

In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Any

# Projekt-Root hinzufügen
project_root = r'C:\Users\sebas\Desktop\coding\PRIM'
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"🔧 Arbeitsverzeichnis: {os.getcwd()}")
print(f"🔧 Projekt-Root: {project_root}")

🔧 Arbeitsverzeichnis: c:\Users\sebas\Desktop\coding\PRIM
🔧 Projekt-Root: C:\Users\sebas\Desktop\coding\PRIM


In [2]:
# Versuche Module aus vorherigen Projekten zu importieren

# Numba-Sieb aus Modul 2
try:
    from modul2_wheel_sieve.modul2_simple_sieve_numba import simple_sieve
    print("✅ Numba-Sieb erfolgreich importiert")
    HAS_NUMBA_SIEVE = True
except ImportError as e:
    print(f"⚠️ Numba-Sieb nicht verfügbar: {e}")
    HAS_NUMBA_SIEVE = False

# FJ32-Test aus Modul 1
try:
    import primetest
    print("✅ FJ32-Test erfolgreich importiert")
    print(f"   Verfügbare Funktionen: {dir(primetest)}")
    HAS_FJ32 = True
except ImportError as e:
    print(f"⚠️ FJ32-Test nicht verfügbar: {e}")
    HAS_FJ32 = False

print(f"\n📊 Status:")
print(f"   Numba-Sieb: {'✅' if HAS_NUMBA_SIEVE else '❌'}")
print(f"   FJ32-Test:   {'✅' if HAS_FJ32 else '❌'}")

✅ Numba-Sieb erfolgreich importiert
✅ FJ32-Test erfolgreich importiert
   Verfügbare Funktionen: ['__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'fj32_c', 'fj_hash_c']

📊 Status:
   Numba-Sieb: ✅
   FJ32-Test:   ✅


## 2. Fallback-Implementierungen

In [3]:
def fallback_simple_sieve(limit: int) -> np.ndarray:
    """
    Fallback-Implementierung des Eratosthenes-Siebs
    falls die Numba-optimierte Version nicht verfügbar ist
    """
    if limit < 2:
        return np.array([], dtype=np.int64)
    
    # Initialisiere Boolean-Array
    is_prime = np.ones(limit, dtype=bool)
    is_prime[0] = is_prime[1] = False
    
    # Sieben
    for i in range(2, int(limit**0.5) + 1):
        if is_prime[i]:
            is_prime[i*i::i] = False
    
    return np.where(is_prime)[0].astype(np.int64)

def fallback_is_prime_fj32(n: int) -> bool:
    """
    Fallback-Implementierung eines einfachen Primzahltests
    falls FJ32 nicht verfügbar ist
    """
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

# Teste Fallback-Funktionen
test_limit = 100
test_primes = fallback_simple_sieve(test_limit)
print(f"📝 Fallback-Sieb Test (N={test_limit}): {len(test_primes)} Primzahlen gefunden")
print(f"   Erste 10: {test_primes[:10]}")

# Teste einige Primzahlen
test_numbers = [17, 25, 97, 100]
print(f"\n📝 Fallback-Primtest:")
for num in test_numbers:
    result = fallback_is_prime_fj32(num)
    print(f"   {num}: {'Prim' if result else 'Nicht prim'}")

📝 Fallback-Sieb Test (N=100): 25 Primzahlen gefunden
   Erste 10: [ 2  3  5  7 11 13 17 19 23 29]

📝 Fallback-Primtest:
   17: Prim
   25: Nicht prim
   97: Prim
   100: Nicht prim


## 3. Hybrid-Analyzer Klasse

Die Hauptklasse für den Hybrid-Workflow kombiniert das optimierte Sieb mit der FJ32-Verifikation.

In [4]:
class HybridPrimeAnalyzer:
    """
    Hauptklasse für den Hybrid-Workflow: Sieb + FJ32-Verifikation
    """
    
    def __init__(self):
        self.results = []
        self.has_numba_sieve = HAS_NUMBA_SIEVE
        self.has_fj32 = HAS_FJ32
        
        print(f"🚀 HybridPrimeAnalyzer initialisiert")
        print(f"   Numba-Sieb: {'✅' if self.has_numba_sieve else '❌ (Fallback)'}")
        print(f"   FJ32-Test:   {'✅' if self.has_fj32 else '❌ (Fallback)'}")
        
    def get_sieve_function(self):
        """Wähle die beste verfügbare Sieb-Funktion"""
        if self.has_numba_sieve:
            return simple_sieve
        else:
            return fallback_simple_sieve
    
    def get_primality_test_function(self):
        """Wähle die beste verfügbare Primzahl-Test-Funktion"""
        if self.has_fj32:
            return lambda n: primetest.is_prime_fj32(int(n))
        else:
            return fallback_is_prime_fj32
        
    def run_sieve_only(self, limit: int):
        """
        Führt nur das Sieb aus und misst die Zeit
        """
        print(f"🔍 Sieb-Test bis {limit:,}...")
        
        sieve_func = self.get_sieve_function()
        
        start_time = time.perf_counter()
        primes = sieve_func(limit)
        end_time = time.perf_counter()
        
        execution_time = end_time - start_time
        print(f"   ✅ {len(primes):,} Primzahlen in {execution_time:.6f}s")
        
        return primes, execution_time
    
    def run_fj32_verification(self, primes):
        """
        Verifiziert Sieb-Primzahlen mit FJ32-Test
        """
        print(f"🔍 FJ32-Verifikation von {len(primes):,} Primzahlen...")
        
        primality_test = self.get_primality_test_function()
        
        verified_count = 0
        failed_count = 0
        failed_numbers = []
        
        start_time = time.perf_counter()
        
        for prime in primes:
            if primality_test(prime):
                verified_count += 1
            else:
                failed_count += 1
                failed_numbers.append(prime)
                if len(failed_numbers) <= 10:  # Zeige max. 10 Fehler
                    print(f"   ⚠️ Widerspruch bei {prime}")
        
        end_time = time.perf_counter()
        execution_time = end_time - start_time
        
        success_rate = (verified_count / len(primes)) * 100 if len(primes) > 0 else 0
        print(f"   ✅ {verified_count:,} bestätigt ({success_rate:.2f}%), {failed_count:,} Widersprüche in {execution_time:.6f}s")
        
        if failed_numbers:
            print(f"   ⚠️ Erste Widersprüche: {failed_numbers[:10]}")
        
        return verified_count, failed_count, execution_time

# Teste die Klasse
analyzer = HybridPrimeAnalyzer()

🚀 HybridPrimeAnalyzer initialisiert
   Numba-Sieb: ✅
   FJ32-Test:   ✅
